In [15]:
!pip install spacy==2.3.5
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz
import re, nltk, spacy, gensim
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from bs4 import BeautifulSoup
from nltk.tokenize import ToktokTokenizer
from nltk.stem import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 
from string import punctuation
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from keras.metrics import top_k_categorical_accuracy
from keras.optimizers import adam_v2
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Conv1D, MaxPooling1D, Flatten
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import pandas as pd
import string
import itertools
import io
import json

     |████████████████████████████████| 12.0 MB 8.0 MB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [17]:
path = "/content/drive/MyDrive/"
pubmed_database = path + "pubmed-diabetesme-set.txt"

#df = pd.read_csv(pubmed_database, sep=" -", header=None, 
                # names=["Terms", "Info"])

#display (df)

In [18]:
def get_PMID_Title_Tabstract_and_mh(lines):
  s = "".join(lines)
  s = re.sub("\n\s+", "", s)
  s = s.strip()
  s = s.split("\n")

  PMID = ()
  title = {}
  abstract = ""
  mh = []

  for line in s:
    if line[0:5]=="PMID-":
      PMID = line[6:]
    elif line[0:5]=="TI  -":
      title = line[6:]
    elif line[0:5]=="AB  -":
      abstract = line[6:]
    elif line[0:5]=="MH  -":
      mh.append(line[6:])
  return [PMID,title,abstract, mh]

In [19]:
records = []
for i, p in enumerate("pubmed-diabetesme-set.txt"):
  f = open(pubmed_database, "r")
  temp_record = []
  r = []
  for line in f:
    if line != "\n":
      r.append(line)
    else :
      temp_record.append(get_PMID_Title_Tabstract_and_mh(r))
      r = []
  f.close()
  records.append(temp_record)

In [ ]:
records[1][0]

['27080137',
 'Diabetes mellitus and its complications in India.',
 "India is one of the epicentres of the global diabetes mellitus pandemic. Rapid socioeconomic development and demographic changes, along with increased susceptibility for Indian individuals, have led to the explosive increase in the prevalence of diabetes mellitus in India over the past four decades. Type 2 diabetes mellitus in Asian Indian people is characterized by a young age of onset and occurrence at low levels of BMI. Available data also suggest that the susceptibility of Asian Indian people to the complications of diabetes mellitus differs from that of white populations. Management of this disease in India faces multiple challenges, such as low levels of awareness, paucity of trained medical and paramedical staff and unaffordability of medications and services. Novel interventions using readily available resources and technology promise to revolutionise the care of patients with diabetes mellitus in India. As ma

In [21]:
records_combined = list(itertools.chain.from_iterable(records))

In [22]:
def one_v_all_learning(records):
  X = []
  y = []
  for d in records:
    X.append(d[0])
    y.append(d[1])
  X = np.sarray(X)
  y = np.sarray(y)
  # Sampling dataset
  vectorizer_X = TfidfVectorizer(analyzer='word', min_df=0.0, max_df = 1.0, 
                                    strip_accents = None, encoding = 'utf-8', 
                                    preprocessor=None, 
                                    token_pattern=r"(?u)\S\S+", # Need to repeat token pattern
                                    max_features=1000)
  # 80/20 split
  X_train, X_test, y_train, y_test = train_test_split(
      X, y, test_size=0.2,train_size=0.8, random_state=0)

  # TF-IDF matrices
  X_tfidf_train = vectorizer_X.fit_transform(X_train)
  X_tfidf_test = vectorizer_X.transform(X_test)

  #dimensionality reduction dengan PCA
  pca = PCA(n_components=0.95)
  X_tfidf_train = pca.fit_transform(X_tfidf_train.todense())
  X_tfidf_test = pca.transform(X_tfidf_test.todense())

  mh_list = []
  for d in records:
    mh_list += d[1]
  values, counts = np.unique(mh_list, return_counts=True)
  ind = (-counts).argsort()[:10]
  top_tags = values[ind]
  print(top_tags)
  reports = {}
  for i in range(len(top_tags)):
    y_train_temp = [1 if top_tags[i] in row else 0 for row in y_train]
    y_test_temp = [1 if top_tags[i] in row else 0 for row in y_test]
    clf = LogisticRegression(penalty= 'elasticnet', solver= 'saga', l1_ratio=0.5, random_state=1)
    clf.fit(X_tfidf_train, y_train_temp)
    preds = clf.predict(X_tfidf_test)
    print("Hasil Klasifikasi Kelas {}".format(top_tags[i]))
    print(classification_report(y_test_temp, preds))
    reports[top_tags[i]] = classification_report(y_test_temp, preds, output_dict=True)
  return reports

In [25]:
logreg_report_combined = one_v_all_learning(records_combined)

AttributeError: ignored